In [ ]:
# backtrader doesn't seem to work?
# pip install backtrader

     |████████████████████████████████| 410 kB 4.1 MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# # two main components to backtrader script
# # strategy class
# # cerebro engine

# import backtrader as bt

# class MyStrategy(bt.Strategy):
#   def next(self):
#     pass # Do something

# #Instantiate Cerebro engine
# cerebro = bt.Cerebro()

# #Add strategy to Cerebro
# cerebro.addstrategy(MyStrategy)

# #Run Cerebro Engine
# cerebro.run()

[]

Cerebro Engine = core of Backtrader
* main class & will add our data and strategies to it before eventually calling the cerebro.run() command

### Get data and import it into Backtrader

1. API from broker to get historical data
2. many 3rd party APIs that allow you to download historical data from within your Python Console
3. Yahoo Finance CSV data download

`log` function allows us to pass in data via the `txt` variable that we want to output to the scren.
- will attempt to grab `datetime` values from the most recent data point, if available, and log it to the screen.
- overwrite `next` function now that printing/logging fn defined.
  - most important part of strategy as most of our code will get executed here.
  - this part gets called every time Backtrader iterates over the next new data point

In [ ]:
# from datetime import datetime
# # Create a subclass of Strategy to define the indicators and logic

# class SmaCross(bt.Strategy):
#     # list of parameters which are configurable for the strategy
#     params = dict(
#         pfast=10,  # period for the fast moving average
#         pslow=30   # period for the slow moving average
#     )

#     def __init__(self):
#         sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
#         sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
#         self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

#     def next(self):
#         if not self.position:  # not in the market
#             if self.crossover > 0:  # if fast crosses slow to the upside
#                 self.buy()  # enter long

#         elif self.crossover < 0:  # in the market & cross to the downside
#             self.close()  # close long position


# cerebro = bt.Cerebro()  # create a "Cerebro" engine instance

# # Create a data feed
# data = bt.feeds.YahooFinanceData(dataname='AMD.csv',
#                                  fromdate=datetime(2011, 1, 1),
#                                  todate=datetime(2012, 12, 31))

# cerebro.adddata(data)  # Add the data feed

# cerebro.addstrategy(SmaCross)  # Add the trading strategy
# cerebro.run()  # run it all
# cerebro.plot()  # and plot it with a single command

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AMD.csv to AMD.csv


In [ ]:
pip install backtesting

     |████████████████████████████████| 175 kB 4.1 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173822 sha256=9bc68c2f9027458841b240fb3824a5c1e4ba09609ac4d1dbbbb59041bde5c33e
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting


--------------------------------------------------

# Backtesting.py

best suited for optimizing position entrance and exit signal strategies, decisions upon values of technical indicators, and it's also a versatile interactive trade visualization and statistics tool.
- bring your own data. Backtesting ingests _all kinds of OHLC data_ (stocks, forex, futures, crypto, ...) as a pandas.DataFrame with columns `'Open'`, `'High'`, `'Low'`, `'Close'` and (optionally) `'Volume'`
- Besides these, your data frames can have additional columns which are accessible in your strategies in a similar manner.

In [ ]:
# Example OHLC daily data for Google Inc.
# 2004 - 2013 data
from backtesting.test import GOOG

GOOG.head()

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900


### Strategy
create first strategy to backtest on these Google data, a SMA cross-over strategy.
- backtesting.py doesn't ship own set of technica analysis indicators.
- users favoring TA should refer to functions from proven indicator libraries, such as `TA-Lib` or `Tulipy`

In [ ]:
import pandas as pd

def SMA(values, n):
  '''
  return SMA of 'values', at each step taking into account `n` previous values.
  '''
  return pd.Series(values).rolling(n).mean()

new strategy needs to extend `Strategy` class and override its two abstract methods: `init()` and `next()`.

method `init()` is invoked before the strategy is run. Within it, one ideally precomputes in efficient, vectorized manner whatever indicators and signlas the strategy depends on 

method `next()` is then iteratively called by the `Backtest` instance once for each data point (data frame row) simulating the incremental availability of each new full candlestick bar.

Note, backtesting cannot make decisions / trades within candlesticks -- any new orders are executed on the next candle's open (or the current candle's close if `trade_on_close = True`). If you find yourself wishing to trade within candlesticks(e.g. day_trading) you instead need to begin with more fine-grained (e.g. hourly) data

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In `init()` as well as `next()`, the data the strategy is simulated on is available as an instance variable `self.data`.

In `init()`, we declare and compute indicators indirectly by wrapping them in `self.I()`. The wrapper is passed a function (our `SMA` function) along with any arguments to call it with (our close values and the MA lag). Indicators wrapped in this way will be automatically plotted, and their legend strings will be intelligently inferred.

In `next()`, we simply check if the faster moving average just crossed over the slower one. If it did and upwards, we close the possible short position and go long; if it did and downwards, we close the open long position and go short. Note, we don't adjust order size, so *Backtesting* assumes *maximal possible position*. We use `backtesting.lib.crossover()` instead of writing more obscure and confusing conditions such as:

In [ ]:
%%script echo

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.position.close()
            self.buy()

        elif (self.sma1[-2] > self.sma2[-2] and    # Ugh!
              self.sma1[-1] < self.sma2[-1]):
            self.position.close()
            self.sell()

In `init()`, the whole series of points was available, whereas in `next()`, the length of `self.data` and all declared indicators is adjusted on each `next()` call so that `array[-1]` (e.g. `self.data.Close[-1]` or `self.sma1[-1]` always  contains the most recent value, `array[-2]` the previous value, etc. (ordinary Python indexing of ascending-sorted 1D arrays).

**NOTE:** `self.data` and any indicators wrapped with `self.I` (e.g. `self.sma1`) are NumPy arrays for performance reasons. If you prefer pandas Series or DataFrame objects, use `Strategy.data.<column>.s` or `Strategy.data.df` accessors respectively. You could also construct the series manually, e.g. `pd.Series(self.data.Close, index = self.data.index)`.

Might avoid `self.position.close()` calls if we primed the `Backtest` instance with `Backtest(..., exclusive_orders = True)`.

## Backtesting

Let's see how our strategy performs on historical Google data. The `Backtest` instance is initialized with OHLC data and a strategy *class* (see API reference for additional options), and we begin with 10,000 units of cash and set broker's commission to realistic 0.2%.

In [ ]:
from backtesting import Backtest

bt = Backtest(GOOG, SmaCross, cash = 10_000, commission = 0.002)
stats = bt.run()
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   97.067039
Equity Final [$]                  68221.96986
Equity Peak [$]                   68991.21986
Return [%]                         582.219699
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   25.266427
Volatility (Ann.) [%]               38.383008
Sharpe Ratio                         0.658271
Sortino Ratio                        1.288779
Calmar Ratio                         0.763748
Max. Drawdown [%]                  -33.082172
Avg. Drawdown [%]                   -5.581506
Max. Drawdown Duration      688 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                   94
Win Rate [%]                        54.255319
Best Trade [%]                       57.11931
Worst Trade [%]                    -16.629898
Avg. Trade [%]                    

`Backtest.run()` method returns a pandas Series of simulation results and statistics associated with our strategy. We see that this simple strategy makes almost 600% return in the period of 9 years, with maximum drawdown of 33% and with longest drawdown period spanning almost 2 years ...

`Backtest.plot()` emthod provides the same insights in a more visual form

In [ ]:
bt.plot()

Row(id='1433', ...)

## Optimization
hard-coded 2 lag parameters `n1` and `n2` into our strategy above. However, the strategy may work better with 15 - 30 or some other cross-over. **Declared parameters as optimizable by making them class variables**.

Optimize the 2 parameters by calling `Backtest.optimize()` method with each parameter a keyword argument pointing to its pool of possible values to test. Parameter `n1` is tested for values in range between 5 and 30 and parameter `n2` for values between 10 and 70, respectively. Some combos of values of the 2 parameters are invalid, i.e. `n1` should not be *larger than* or equal to `n2`. Limit admissible parameter combos with an *ad hoc* constraint function, which takes in the parameters and returns `True` (i.e. admissible) whenever `n1` is less than `n2`. Additionally, search for such parameter combo that maximizes return over the observed period. We could instead choose to optimize any other key from the returned `stats` series.

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 474 ms, sys: 53.1 ms, total: 527 ms
Wall time: 7.22 s


In [ ]:
stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   99.068901
Equity Final [$]                 103949.42612
Equity Peak [$]                  108327.71798
Return [%]                         939.494261
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   31.610936
Volatility (Ann.) [%]               44.739816
Sharpe Ratio                          0.70655
Sortino Ratio                        1.490961
Calmar Ratio                         0.718505
Max. Drawdown [%]                  -43.995445
Avg. Drawdown [%]                   -6.138853
Max. Drawdown Duration      690 days 00:00:00
Avg. Drawdown Duration       43 days 00:00:00
# Trades                                  153
Win Rate [%]                        51.633987
Best Trade [%]                      61.562908
Worst Trade [%]                    -19.778312
Avg. Trade [%]                    

In [ ]:
# n1 = 10, n2 = 15
stats._strategy

<Strategy SmaCross(n1=10,n2=15)>

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

Row(id='2029', ...)

Strategy optimization managed to up its initial performance *on in-sample data* by almost 50% and even beat simple buy & hold. In real life optimization, however, do **take steps to avoid overfitting**

## Trade data

In addition to backtest statistics returned by `Backtest.run()` shown above, you can look into *individual trade returns* and the changing equity curve and drawdown by inspecting last few internal keys in the result series.

In [ ]:
stats.tail()

Expectancy [%]                                              1.97988
SQN                                                        1.604158
_strategy                                     SmaCross(n1=10,n2=15)
_equity_curve                       Equity  DrawdownPct Drawdown...
_trades                Size  EntryBar  ExitBar  EntryPrice  Exit...
dtype: object

In [ ]:
stats['_equity_curve']  # Contains equity/drawdown curves. DrawdownDuration is only defined at ends of DD periods.

,Equity,DrawdownPct,DrawdownDuration
2004-08-19,10000.00000,0.000000,NaT
2004-08-20,10000.00000,0.000000,NaT
2004-08-23,10000.00000,0.000000,NaT
2004-08-24,10000.00000,0.000000,NaT
2004-08-25,10000.00000,0.000000,NaT
...,...,...,...
2013-02-25,103035.52612,0.048854,NaT
2013-02-26,102952.32612,0.049622,NaT
2013-02-27,104206.82612,0.038041,NaT
2013-02-28,104391.42612,0.036337,NaT


In [ ]:
stats['_trades']  # Contains individual trade data

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,87,20,60,114.64884,185.23,6140.56092,0.615629,2004-09-17,2004-11-12,56 days
1,-87,60,69,184.85954,175.80,788.17998,0.049008,2004-11-12,2004-11-26,14 days
2,96,69,71,176.15160,180.71,437.60640,0.025878,2004-11-26,2004-11-30,4 days
3,-96,71,75,180.34858,179.13,116.98368,0.006757,2004-11-30,2004-12-06,6 days
4,97,75,82,179.48826,177.99,-145.33122,-0.008347,2004-12-06,2004-12-15,9 days
...,...,...,...,...,...,...,...,...,...,...
148,139,2085,2111,689.15556,735.54,6447.43716,0.067306,2012-11-29,2013-01-08,40 days
149,-139,2111,2113,734.06892,742.83,-1217.79012,-0.011935,2013-01-08,2013-01-10,2 days
150,136,2113,2121,744.31566,735.99,-1132.28976,-0.011186,2013-01-10,2013-01-23,13 days
151,-136,2121,2127,734.51802,750.51,-2174.90928,-0.021772,2013-01-23,2013-01-31,8 days


In [ ]:
GOOG

,Open,High,Low,Close,Volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900
...,...,...,...,...,...
2013-02-25,802.30,808.41,790.49,790.77,2303900
2013-02-26,795.00,795.95,784.40,790.13,2202500
2013-02-27,794.80,804.75,791.11,799.78,2026100
2013-02-28,801.10,806.99,801.03,801.20,2265800


In [ ]:
data = pd.read_csv('AMD.csv')

In [ ]:
AMD = data

In [ ]:
btamd = Backtest(AMD, SmaCross, cash = 10_000, commission = 0.002)
stats = btamd.run()
stats

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  """Entry point for launching an IPython kernel.


Start                                     0.0
End                                    1811.0
Duration                               1811.0
Exposure Time [%]                   98.785872
Equity Final [$]                  4127.239001
Equity Peak [$]                    14186.8003
Return [%]                          -58.72761
Buy & Hold Return [%]             3805.992547
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -88.861181
Avg. Drawdown [%]                  -19.057982
Max. Drawdown Duration                 1692.0
Avg. Drawdown Duration             297.166667
# Trades                                101.0
Win Rate [%]                        41.584158
Best Trade [%]                      70.608843
Worst Trade [%]                    -38.739016
Avg. Trade [%]                    

In [ ]:
%%time

statsamd = btamd.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
statsamd

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 197 ms, sys: 36.5 ms, total: 233 ms
Wall time: 5.78 s


In [ ]:
statsamd._strategy

<Strategy SmaCross(n1=5,n2=60)>

In [ ]:
# simple SMA crossover strategy optimization doesn't beat Buy and hold for AMD
statsamd

Start                                     0.0
End                                    1811.0
Duration                               1811.0
Exposure Time [%]                   93.543046
Equity Final [$]                101972.103678
Equity Peak [$]                 105533.423034
Return [%]                         919.721037
Buy & Hold Return [%]             3805.992547
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -58.313086
Avg. Drawdown [%]                  -12.451527
Max. Drawdown Duration                  377.0
Avg. Drawdown Duration              38.790698
# Trades                                 50.0
Win Rate [%]                             38.0
Best Trade [%]                     143.157752
Worst Trade [%]                    -20.847959
Avg. Trade [%]                    

# Library of Composable base strategies



In [ ]:
from backtesting.test import SMA

Part of this software distribution is `backtesting.lib` module that contains various reusable utilities for strategy development. Some of those utilities are composable base strategies we can extend and build upon.

We import and extend 2 of those strategies here:
- `SignalStrategy` which decides upon a single signal vector whether to buy into a position, akin to vectorized backtesting engines, and 
- `TrailingStrategy` which auto trails the current price with a stop-loss order some multiple of average true range (ATR) away.?}

In [ ]:
import pandas as pd
from backtesting.lib import SignalStrategy, TrailingStrategy

class SmaCross(SignalStrategy, TrailingStrategy):
  n1 = 10
  n2 = 25

  def init(self):
    # In init() and in next() it is important to call the
    # super method to properly initialize the parent classes
    super().init()
     
    # Precompute the two moving averages
    sma1 = self.I(SMA, self.data.Close, self.n1)
    sma2 = self.I(SMA, self.data.Close, self.n2)

    # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
    signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
    signal = signal.replace(-1, 0)  # Upwards/long only

    # Use 95% of available liquidity (at the time) on each order.
    # (Leaving a value of 1. would instead buy a single share.)
    entry_size = signal * .95
                
    # Set order entry sizes using the method provided by 
    # `SignalStrategy`. See the docs.
    self.set_signal(entry_size=entry_size)
        
    # Set trailing stop-loss to 2x ATR using
    # the method provided by `TrailingStrategy`
    self.set_trailing_sl(2)

In [ ]:
# from backtesting import Backtest
# from backtesting.test import GOOG

bt = Backtest(GOOG, SmaCross, commission=.002)

bt.run()
bt.plot()

Row(id='2721', ...)

In [ ]:
bt.run()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   29.515829
Equity Final [$]                   31874.8637
Equity Peak [$]                   38202.59246
Return [%]                         218.748637
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   14.568116
Volatility (Ann.) [%]               18.649301
Sharpe Ratio                         0.781162
Sortino Ratio                        1.657039
Calmar Ratio                         0.623338
Max. Drawdown [%]                  -23.371122
Avg. Drawdown [%]                   -4.003572
Max. Drawdown Duration      674 days 00:00:00
Avg. Drawdown Duration       74 days 00:00:00
# Trades                                   38
Win Rate [%]                        55.263158
Best Trade [%]                      48.410457
Worst Trade [%]                     -9.055262
Avg. Trade [%]                    

# Multiple Time Frames

In [ ]:
def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()


def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

Strategy:
Buy when:
- weekly RSI(30) >= daily RSI(30) > 70
- Close > MA(10) > MA(20) > MA(50) > MA(100)

Close Position when:
- Daily close is more than 2% *below* MA(10)
- 8% fixed stop loss is hit

Need to provide bars data in the lowest time frame (i.e. daily) and resample it to any higher time frame (i.e. weekly) that our strategy requires.

In [ ]:
from backtesting import Strategy, Backtest
from backtesting.lib import resample_apply


class System(Strategy):
    d_rsi = 30  # Daily RSI lookback periods
    w_rsi = 30  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma20 = self.I(SMA, self.data.Close, 20)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        
        # Compute daily RSI(30)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and
            self.daily_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.level and
            self.weekly_rsi[-1] > self.daily_rsi[-1] and
            self.ma10[-1] > self.ma20[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 8% fixed stop loss.
            self.buy(sl=.92 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .98 * self.ma10[-1]:
            self.position.close()

In [ ]:
backtest = Backtest(GOOG, System, commission=.002)
backtest.run()

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    2.793296
Equity Final [$]                  10017.44422
Equity Peak [$]                    10978.3801
Return [%]                           0.174442
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                     0.02045
Volatility (Ann.) [%]                4.941212
Sharpe Ratio                         0.004139
Sortino Ratio                         0.00536
Calmar Ratio                         0.002043
Max. Drawdown [%]                   -10.00745
Avg. Drawdown [%]                   -9.340092
Max. Drawdown Duration     2653 days 00:00:00
Avg. Drawdown Duration     1410 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       9.687579
Worst Trade [%]                     -4.456159
Avg. Trade [%]                    

In [ ]:
%%time

backtest.optimize(d_rsi=range(10, 35, 5),
                  w_rsi=range(10, 35, 5),
                  level=range(30, 80, 10))

Backtest.optimize:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 331 ms, sys: 66 ms, total: 396 ms
Wall time: 15.9 s


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   22.486034
Equity Final [$]                  22587.83224
Equity Peak [$]                   23395.59144
Return [%]                         125.878322
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    10.03124
Volatility (Ann.) [%]               13.124247
Sharpe Ratio                         0.764329
Sortino Ratio                         1.28711
Calmar Ratio                         0.530172
Max. Drawdown [%]                  -18.920719
Avg. Drawdown [%]                   -3.795058
Max. Drawdown Duration      778 days 00:00:00
Avg. Drawdown Duration       97 days 00:00:00
# Trades                                   23
Win Rate [%]                        65.217391
Best Trade [%]                      25.034669
Worst Trade [%]                     -6.297769
Avg. Trade [%]                    

In [ ]:
backtest.plot()

Row(id='3664', ...)

In conclusion, to test strategies on multiple time frames, you need to pass in OHLC data in the lowest time frame, then resample it to higher time frames, apply the indicators, then resample back to the lower time frame, filling in the in-betweens. Which is what the function `backtesting.lib.resample_apply()` does for you.